In [1]:
%cd -q ..

import numpy as np
import pandas as pd

from model.helpers import load_params
from model.model import Model
from model.inpatients import InpatientsModel
from model.outpatients import OutpatientsModel
from model.aae import AaEModel
from model.health_status_adjustment import HealthStatusAdjustmentInterpolated

pd.set_option("display.precision", 1)

In [2]:
import logging

logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [3]:
params = load_params("queue/sample_params.json")
run_params = Model.generate_run_params(params)
hsa = HealthStatusAdjustmentInterpolated(
    f"data/2019/{params['dataset']}", params["start_year"], params["end_year"]
)

# Helper Methods

Below are some helper methods for turning the results into useful Pandas `DataFrames`.

In [4]:
def run_model(model, i=0):
    return {
        k0: pd.DataFrame([{**dict(k1), "value": v1} for k1, v1 in v0.items()])
        for k0, v0 in model(params, "data", hsa, run_params).go(i).items()
    }

In [5]:
def default_pod_measure_pivot(agg):
    default_results = (
        agg["default"]
        .groupby(["pod", "measure"], as_index=False)
        .agg({"value": sum})
        .pivot(index=["pod"], columns="measure")
        .fillna(0)
    )
    default_results.loc["total"] = default_results.sum()

    return default_results

In [6]:
def step_counts_simple(agg):
    step_counts = (
        agg["step_counts"]
        .drop(columns=["strategy", "activity_type"])
        .groupby(["change_factor", "measure"], as_index=False)
        .sum()
        .pivot(index="change_factor", columns="measure")
        .loc[agg["step_counts"]["change_factor"].unique()]
    )
    step_counts.loc["total"] = step_counts.sum()

    return step_counts

# Running the models

## Inpatients

Running the principal model run (model run 0) for the Inpatients data

In [7]:
ip_agg = run_model(InpatientsModel)

### Aggregate Results

The available aggregations for inpatients are:

In [8]:
list(ip_agg.keys())

['default',
 'sex+age_group',
 'sex+tretspef',
 'bed_occupancy',
 'theatres_available',
 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [9]:
default_pod_measure_pivot(ip_agg)

value                                 
measure                   admissions attendances   beddays procedures
pod                                                                  
ip_elective_admission         7286.0         0.0   29382.0     4039.0
ip_elective_daycase          55725.0         0.0   55725.0    30098.0
ip_maternity_admission        7204.0         0.0   32716.0     3915.0
ip_non-elective_admission    45561.0         0.0  163311.0     6138.0
op_procedure                     0.0      1048.0       0.0        0.0
total                       115776.0      1048.0  281134.0    44190.0

### Estimate of effects

In [10]:
step_counts_simple(ip_agg)

value          
measure                    admissions   beddays
change_factor                                  
baseline                      97536.0  309343.0
demographic_adjustment        19630.4   60745.0
health_status_adjustment        246.5     885.4
covid_adjustment               1644.5    4195.0
expat                          -214.8    -717.6
repat                          2402.1    7643.3
waiting_list_adjustment         166.4     226.4
baseline_adjustment               0.0       0.0
non-demographic_adjustment    12267.3   39003.5
activity_avoidance           -16854.3  -70707.1
efficiencies                  -1048.0  -69483.0
total                        115776.0  281134.0

## Outpatients

Running the principal model run (model run 0) for the Outpatients data

In [11]:
op_agg = run_model(OutpatientsModel)

### Aggregate Results

The available aggregations for outpatients are:

In [12]:
list(op_agg.keys())

['default', 'sex+age_group', 'sex+tretspef', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [13]:
default_pod_measure_pivot(op_agg)

value                 
measure      attendances tele_attendances
pod                                      
op_first          155280            17708
op_follow-up      308198            37798
op_procedure       41825                0
total             505303            55506

### Estimate of effects

In [14]:
step_counts_simple(op_agg)

value                 
measure                  attendances tele_attendances
change_factor                                        
baseline                    440885.0          14276.0
demographic_adjustment       75376.3           2664.0
health_status_adjustment       603.5             19.2
covid_adjustment             55091.6           1684.3
expat                        -1467.6            -48.6
repat                        11398.3            361.4
waiting_list_adjustment       7887.5            250.8
baseline_adjustment              0.0              0.0
activity_avoidance          -46703.7          -1469.1
efficiencies                -37768.0          37768.0
total                       505303.0          55506.0

## Accident and Emergency

Running the principal model run (model run 0) for the A&E data

In [15]:
ae_agg = run_model(AaEModel)

### Aggregate Results

The available aggregations for outpatients are:

In [16]:
list(ae_agg.keys())

['default', 'sex+age_group', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [17]:
default_pod_measure_pivot(ae_agg)

value        
measure     ambulance walk-in
pod                          
aae_type-01     36384   68928
aae_type-02         3    1304
aae_type-03       276   15934
aae_type-04         0    1262
total           36663   87428

### Estimate of effects

In [18]:
step_counts_simple(ae_agg)

,value
measure,arrivals
change_factor,
baseline,116518.0
demographic_adjustment,13770.7
health_status_adjustment,76.7
covid_adjustment,3579.5
expat,-2743.5
repat,2688.6
baseline_adjustment,0.0
activity_avoidance,-9799.0
